# Reading Multi Spectral Images

https://nbviewer.jupyter.org/github/thomasaarholt/hyperspy-demos/blob/master/2_SVD_and_BSS.ipynb

## Bands and Wavelengths
When talking about spectral data, we talk from both, the electromagnetic spectrum and image bands. Spectral remote sensing data are collected by powerful camera-like instruments known as imaging spectrometers. Imaging spectrometers collect reflected light energy in “bands.”

A band represents a segment of the electromagnetic spectrum. For example, the wavelength values between 800 nanometers (nm) and 850 nm might be one band captured by an imaging spectrometer. The imaging spectrometer collects reflected light energy within a pixel area on the ground. Since an imaging spectrometer collects many different types of light - for each pixel the amount of light energy for each type of light or band will be recorded. So, for example, a camera records the amount of red, green and blue light for each pixel.

Often when we work with a multispectral dataset, the band information is reported as the center wavelength value. This value represents the center point value of the wavelengths represented in that band. Thus in a band spanning 800-850 nm, the center would be 825 nm.

## Spectral Resolution
The spectral resolution of a dataset that has more than one band, refers to the spectral width of each band in the dataset. While a general spectral resolution of the sensor is often provided, not all sensors collect information within bands of uniform widths.

## Spatial Resolution
The spatial resolution of a raster represents the area on the ground that each pixel covers. If you have smaller pixels in a raster the data will appear more “detailed.” If you have large pixels in a raster, the data will appear more coarse or “fuzzy.”


## Multispectral Imagery
Images obtained with a ADC Lite - Tetracam's Lightweight ADC

I made pitures about: 

>Aluminum , Copper, Brass, Iron, Stainless Steel, Painted Iron 

http://tetracam.com/Products-ADC_Lite.htm

MRobalinho - 25-03-2019

In [ ]:
# Some libraries
import glob, os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
from openpyxl import load_workbook

In [ ]:
# Verify my current folder
currDir = os.path.dirname(os.path.realpath("__file__"))
mypath = currDir
print(currDir)  

In [ ]:
# Path to the image files
folder = "imagedata04"
path = currDir + "/" + folder + "/"

# Part name of file to filter files
end_file = "TIF"

In [ ]:
# Read files from folder
print(' ---- IMAGES ON THE FOLDER -------')
for file in os.listdir(path):
    if file.endswith(end_file):
        print(os.path.join(file))

In [ ]:
# Create Data Frame with image information
df_image = []

In [ ]:
# Look from an chanel from then image

def channel(img, n):
    """Isolate the nth channel from the image.

       n = 0: red, 1: green, 2: blue
    """
    a = np.array(img)
    a[:,:,(n!=0, n!=1, n!=2)] *= 0
#    a[:,:,n] *= 0
    print(Image.fromarray(a), 'Get Channel n: ', n)
  
    return Image.fromarray(a)

# def to resize 
# Given parameters : image , number to divide (resize)
def imageResize(img, n):
    width, height = img.size 

    print('Original size:', width, '/', height, 'Resize:',n)
    
    newWidth = int(width / n)
    newHeight = int(height / n)
    img.resize((newWidth, newHeight), Image.ANTIALIAS)
    print('New size:', newWidth, '/', newHeight)
    return img

In [ ]:
def print_file(path, xfile):
  print('------------------------------------------------------------------------')   
  tif_f1 = Image.open(path+xfile)
    
  # Transform Image to array
  aArray = np.array(tif_f1)
  # Array sum  
  xsum = aArray.sum() / 1000000
    
  # Get channel 0
  x0_channel = channel(tif_f1, 0)
  aArray = np.array(x0_channel)
  xsum_0 = aArray.sum() / 1000000  
    
  # Get channel 1
  x1_channel = channel(tif_f1, 1)
  aArray = np.array(x1_channel)
  xsum_1 = aArray.sum() / 1000000  
         
  # Get channel 2
  x2_channel = channel(tif_f1, 2)
  aArray = np.array(x2_channel)
  xsum_2 = aArray.sum() / 1000000  

  # Histogram from image
  aHist = tif_f1.histogram()
  hsum = sum(aHist) / 100000

  # Histogram channel 0
  aHist_0 = x0_channel.histogram()
  hsum_0 = sum(aHist_0) / 100000

  # Histogram channel 1
  aHist_1 = x1_channel.histogram()
  hsum_1 = sum(aHist_1) / 100000
  
  # Histogram chanel 0
  aHist_2 = x2_channel.histogram()
  hsum_2 = sum(aHist_2) / 100000
 
  # number elements on list
  nlist = len(aHist)
    
  # Get color
  pix_val = list(tif_f1.getdata())
  # Transform tuple in a list
  pix_val_flat = [x for sets in pix_val for x in sets]
  # Sum the list and medium list pixel
  sum_pix = sum(pix_val_flat)
  med_pix = sum_pix / len(pix_val_flat)

  # Obtain name file without extension 
  sample_name = os.path.basename(xfile).split('_')[0]
 
  # Print information  
  print('Inf.File:',xfile , sample_name,' Size:',tif_f1.size, ' Format:',tif_f1.format, ' Mode:', tif_f1.mode)
  # More information image
  print('         ',' Bands Extrems:',tif_f1.getextrema())
  # More information image
  print('         ',' Sum array:',xsum, ' Sum Ch 0:', xsum_0, ' Sum Ch 1:', xsum_1, ' Sum Ch 2:', xsum_2)      
  # More information image
  print('         ',' Histog:', hsum ,' Histog Ch0:', hsum_0,' Histog Ch1:', hsum_1,' Histog Ch2:', hsum_2, '  N.List elem:', nlist)
  print('         ',' Color:', med_pix)  
  # insert information in a Pandas Data Frame
  df_image.append((folder, xfile, sample_name, tif_f1.size, tif_f1.format, tif_f1.mode ,tif_f1.getextrema(), 
                   xsum, xsum_0, xsum_1, xsum_2, hsum, hsum_0,hsum_1,hsum_2, nlist, med_pix))  

In [ ]:
# Create Data Frame with image information
df_image = []

xend_file = "*" + end_file
os.chdir(path)
for file in glob.glob(xend_file):
 #   print(file)
    print_file(path,file)

In [ ]:
df = pd.DataFrame(df_image,columns=['Folder','File','Material','Size','Format','Mode','Bands Extrems',
                                    'Array_sum', 'Sum_Ch0','Sum_Ch1','Sum_Ch2',
                                    'Histogram','Hist_Ch0','Hist_Ch1','Hist_Ch2','Number_list_elements','Color'])
df.head()

In [ ]:
# Verify my current folder
path = mypath + r"/upt_data.xlsx"
print('Write statistics into file :', path)

# Block to Read excel old excel file
book = load_workbook(path)
writer = pd.ExcelWriter(path, engine = 'openpyxl')
writer.book = book
# ------------------------

# Write statistics into excel file
#writer = pd.ExcelWriter(path, engine = 'xlsxwriter') # only for new excelfile
df.to_excel(writer, sheet_name = folder)
writer.save()
writer.close()

In [ ]:
df_plot = pd.DataFrame(df, columns=["Material", "Array_sum", "Sum_Ch0", "Sum_Ch1", "Sum_Ch2","Color"])
df_plot


In [ ]:
df_plot.Sum_Ch0 = df_plot.Sum_Ch0 + 100 # to have diference lines during plot
df_plot.Sum_Ch1 = df_plot.Sum_Ch1 + 200
df_plot.Sum_Ch2 = df_plot.Sum_Ch2 + 300
df_plot.Color  = df_plot.Color * 10
df_plot

In [ ]:
df_plot.plot(y=["Array_sum","Sum_Ch0","Sum_Ch1", "Sum_Ch2","Color"],figsize=(10,5), grid=True )

# Obtain legend (xticks) for X axis
loc_Array_sum = np.arange(len(df_plot.index))
# Position of X labels
xtick_loc = list(loc_Array_sum)  
# Name of x labels
xticks = list(df_plot.Material)
#-------

#plt.plot(df_plot.Array_sum)
plt.title('IMAGE WITH ALL CHANNELS',fontsize=20)
plt.ylabel('Sum of image matrix',fontsize=18)
plt.xticks(xtick_loc, df_plot.Material, rotation=0)
plt.xlabel('Material',fontsize=18)
plt.show()

In [ ]:
# Create pivot table
df_plot1 = df_plot.groupby('Material')['Array_sum', 'Sum_Ch0','Sum_Ch1','Sum_Ch2','Color'].mean()
df_plot1

In [ ]:
df = pd.DataFrame(df_plot1.Array_sum)   
color = ['red','blue','green','orange','cyan','black']

In [ ]:
df.plot(kind='bar', y=0, color=color, legend=False, rot=0, figsize=(10,5))
plt.title('IMAGE WITH ALL CHANNELS',fontsize=20)
plt.xlabel('Material',fontsize=18)
plt.ylabel('Sum of image matrix',fontsize=18)
plt.show()

In [ ]:
loc_Array_sum = np.arange(len(df_plot1.index))
xtick_loc = list(loc_Array_sum)  
xticks = list(df_plot1.index)

df_plot1.plot( y=["Array_sum","Sum_Ch0","Sum_Ch1", "Sum_Ch2","Color"],figsize=(10,5), grid=True )
plt.xticks(xtick_loc, df_plot1.index, rotation=0)
plt.title('IMAGE WITH ALL CHANNELS',fontsize=20)
plt.xlabel('Material',fontsize=18)
plt.ylabel('Sum of image matrix',fontsize=18)
plt.show()

In [ ]:
loc_Array_sum = np.arange(len(df_plot1.index))+0.1 # Offsetting the tick-label location
loc_r = np.arange(len(df_plot1.index))-0.1 # Offsetting the tick-label location
loc_g = np.arange(len(df_plot1.index))-0.3 # Offsetting the tick-label location
loc_b = np.arange(len(df_plot1.index))-0.5 # Offsetting the tick-label location

xtick_loc = list(loc_g)  
xticks = list(df_plot1.index)

In [ ]:
#Plot  Bar Graph
#df_plot1.plot(kind='bar', figsize=(12,5), grid=True, color='darkred',fontsize=18)
loc_Array_sum = np.arange(len(df_plot1.index))+0.1 # Offsetting the tick-label location
loc_b = np.arange(len(df_plot1.index))-0.1 # Offsetting the tick-label location
loc_g = np.arange(len(df_plot1.index))-0.3 # Offsetting the tick-label location
loc_r = np.arange(len(df_plot1.index))-0.5 # Offsetting the tick-label location

#xtick_loc = list(loc_Array_sum) + list(loc_r) + list(loc_g) + list(loc_b)
#xticks = list(selected.keys())+ list(rejected.keys())
colors = ['darkred','red','green','blue','orange','cyan','black']
plt.figure(figsize=(12,5))

plt.bar(loc_Array_sum, df_plot1.Array_sum, color=colors[0], width=0.2, label='Array Sum')
plt.bar(loc_r, df_plot1.Sum_Ch0, color=colors[1], width=0.2,label='Band R')
plt.bar(loc_g, df_plot1.Sum_Ch1, color=colors[2], width=0.2,label='Band G')
plt.bar(loc_b, df_plot1.Sum_Ch2, color=colors[3], width=0.2,label='Band B')

plt.title('IMAGE WITH ALL CHANNELS',fontsize=20)
plt.xlabel('Material',fontsize=18)
plt.ylabel('Sum of image matrix',fontsize=18)
plt.xticks(xtick_loc, xticks, rotation=0)
plt.legend(bbox_to_anchor=(.8,0.8),\
    bbox_transform=plt.gcf().transFigure)

plt.show()

In [ ]:
plt.figure(1)
plt.figure(figsize=(17, 4))
plt.tight_layout()
plt.subplot(231)
plt.title('IMAGE CHANNEL 0')
plt.xticks(rotation=45)
plt.grid(True)
plt.plot(df_plot1.Sum_Ch0, 'k--')

plt.subplot(232)
plt.title('IMAGE CHANNEL 1')
plt.xticks(rotation=45)
plt.grid(True)
plt.plot(df_plot1.Sum_Ch1,  'r--')

plt.subplot(233)
plt.title('IMAGE CHANNEL 2')
plt.xticks(rotation=45)
plt.plot(df_plot1.Sum_Ch2,  'g--')
plt.grid(True)
plt.show()

In [ ]:
# Plot channel based
plt.plot(df_plot1.Array_sum)
plt.title('IMAGE WITH ALL CHANNELS',fontsize=20)
plt.xlabel('Material',fontsize=18)
plt.ylabel('Sum of image matrix',fontsize=18)
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

In [ ]:
# Plot based on color
plt.plot(df_plot1.Color)
plt.title('IMAGE WITH COLORS BASED',fontsize=20)
plt.xlabel('Material',fontsize=18)
plt.ylabel('Sum of color image',fontsize=18)
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

# Using PIL to read images


In [ ]:
# Path to the image files
folder = "imagedata04"
path = mypath + "/" + folder + "/"
path

In [ ]:
os.chdir(path)

In [ ]:
# Read TIFF Image
from PIL import Image

f1_alum = 'Copper_1.TIF'
tif_f1_alum = Image.open(f1_alum)
#
#print('File name:',f1_alum , '     Size File:',tif_f1_alum.size, '    Format file:',tif_f1_alum.format)
tif_f1_alum

In [ ]:
# Print Bands from image
tif_f1_alum.getbands()

In [ ]:
# Display extrems from each band
def print_bands(path, xfile):
    tif_f1 = Image.open(path+xfile)
    print('Bands Extrems from File name:',xfile ,tif_f1.getextrema())    

In [ ]:
mypath

In [ ]:
# Read files from folder - ALUMINUM

for file in glob.glob("*.tif"):
 #   print(file)
    print_bands(path,file)

In [ ]:
print(tif_f1_alum.getcolors())

In [ ]:
# Transform Image to array
aArray = np.array(tif_f1_alum)
#aArray = np.array(x0_channel)
#a[:,:,0] *=0
#a[:,:,1] *=0

# Transform array to Image
cImage = Image.fromarray(aArray)
aArray 

In [ ]:
# Transform array to list
matrix = aArray.tolist()
#print("Original array: "+str(matrix))
matrix

In [ ]:
# Soma do array da imagem
xsum = aArray.sum()
xsum

In [ ]:
# Get channel 0
x0_channel = channel(cImage, 0)
x0_channel

In [ ]:
# Get channel 1
x1_channel = channel(cImage, 1)
x1_channel

In [ ]:
# Get channel 2
x2_channel = channel(cImage, 2)
x2_channel

In [ ]:
# Resize Image
aFile = cImage
xFile = imageResize(aFile, 20)
   
          
#Saved in the same relative location 
xFile.save("resized_picture.jpg") 

In [ ]:
print(xFile)

In [ ]:
# Getting histogram of image 
aHistgram = aFile.histogram()
print(aHistgram)

In [ ]:
print(sum(aHistgram), len(aHistgram))

In [ ]:
# Getting histogram of image of channel 0 - R
aHist_0= x0_channel.histogram() 
print(aHist_0)

In [ ]:
print(sum(aHist_0), len(aHistgram))

In [ ]:
# Getting histogram of image of channel 1 - B
aHist_1= x1_channel.histogram() 
print(aHist_1)

In [ ]:
# Getting histogram of image of channel 2 - B 
aHist_2= x2_channel.histogram() 
print(aHist_2)

In [ ]:
plt.plot(aHistgram)
plt.title('IMAGE WITH ALL CHANNELS')
plt.ylabel('My Image for all channels')
plt.show()

In [ ]:
plt.plot(aHist_0, color='r')
plt.title('IMAGE CHANNEL 0')
plt.ylabel('My Image channel 0')
plt.show()

In [ ]:
plt.plot(aHist_1, color='g')
plt.title('IMAGE CHANNEL 1')
plt.ylabel('My Image channel 1')
plt.show()

In [ ]:
plt.plot(aHist_2, color='b')
plt.title('IMAGE CHANNEL 2')
plt.ylabel('My Image channel 2')
plt.show()

In [ ]:
plt.figure(1)
plt.figure(figsize=(17, 4))
plt.tight_layout()
plt.subplot(231)
plt.title('IMAGE CHANNEL 0')
plt.plot(aHist_0,   'k--')

plt.subplot(232)
plt.title('IMAGE CHANNEL 1')
plt.plot(aHist_1,  'r--')

plt.subplot(233)
plt.title('IMAGE CHANNEL 2')
plt.plot(aHist_2,  'g--' )
plt.show()

In [ ]:
## Using Matplotlib

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

In [ ]:
# Read in the image and print some stats
file1 = f1_alum
image = mpimg.imread(file1)
print(f1_alum, '   This image is: ',type(image), 
     'with dimensions:', image.shape)

In [ ]:
# Plotting the image

imgplot=plt.imshow(image)
plt.show()# Using Matplotlib to plot the Image

In [ ]:
#splitting the image 
print(file1.split()) 

In [ ]:
bFile = file1.split()
bFile

In [ ]:
#In-place modification 
cFile = cImage.thumbnail((200, 200)) 
cImage

In [ ]:
print(tiff.mode) 
          
#converting image to bitmap 
print(tiff.tobitmap()) 
          
print(type(tiff.tobitmap()) )

In [ ]:
# Transform image in an array
import numpy as np
imarray = np.array(image)

In [ ]:
# Information array image
imarray.shape

In [ ]:
# Information array image
image.size

In [ ]:
Image.fromarray(imarray)

In [ ]:
from PIL import Image, ImageFile
import string


In [ ]:
from PIL import Image, ImageFilter
try:
    original = Image.open(file1)
except:
    print ("Unable to load image")


In [ ]:
original

In [ ]:
print ("The size of the Image is: ")
print(original.format, original.size, original.mode)

In [ ]:
# https://www.pythonforbeginners.com/gui/how-to-use-pillow
# The Pillow module provides the following set of predefined image enhancement filters:

# BLUR
# CONTOUR
# DETAIL
# EDGE_ENHANCE
# EDGE_ENHANCE_MORE
# EMBOSS
# FIND_EDGES
# SMOOTH
# SMOOTH_MORE
# SHARPEN

In [ ]:
# Blur the image
blurred = original.filter(ImageFilter.BLUR)

# Display both images
original.show()

blurred.show()

# save the new image
blurred.save("blurred.png")

In [ ]:
# Creating Thumbnails
size = (128, 128)
saved = file1
try:
    im =  Image.open(file1)
except:
    print ("Unable to load image")

im.thumbnail(size)
im.save(saved)
im.show()

In [ ]:
# Filter SMOOTH the image
filter_im = original.filter(ImageFilter.SMOOTH_MORE)

#filgter_im.show()

In [ ]:
filter_im

In [ ]:
# Obtain R,G,B separately
r, g, b = filter_im.split()

In [ ]:
r

In [ ]:
g

In [ ]:
b

In [ ]:
imarray = np.array(r)
imarray

In [ ]:
imarray = np.array(g)
imarray

In [ ]:
imarray = np.array(b)
imarray

In [ ]:
# Convert into modes
im = Image.open(file1).convert("L")

In [ ]:
im

In [ ]:
print(im.format, im.size, im.mode)

In [ ]:
# split the image into individual bands
source = filter_im.split()

R, G, B = 0, 1, 2

# select regions where red is less than 100
mask = source[R].point(lambda i: i < 100 and 255)

# process the green band
out = source[G].point(lambda i: i * 0.7)

# paste the processed band back, but only where red was < 100
b = source[G].paste(out, None, mask)

# build a new multiband image
im = Image.merge(filter_im.mode, source)

In [ ]:
source[G]

In [ ]:
# Enhancing images

In [ ]:
from PIL import ImageEnhance

enh = ImageEnhance.Contrast(im)
enh.enhance(1.3).show("30% more contrast")

In [ ]:
# Verify my current folder
currDir = os.path.dirname(os.path.realpath("__file__"))
mypath = currDir
print(currDir)  

In [ ]:
# Path to the image files
#folder = "imagedata03"
#path = currDir + "/" + folder + "/"

In [ ]:
from PIL import Image

xfile = mypath + '\Copper_2.jpg'

im = Image.open(xfile) # Can be many different formats.
pix = im.load()
print (im.size)  # Get the width and hight of the image for iterating over
print (pix[100,100])  # Get the RGBA Value of the a pixel of an image
#pix[x,y] = value  # Set the RGBA Value of the image (tuple)

im.save('alive_parrot.png')  # Save the modified pixels as .png

In [ ]:
from PIL import Image
img = Image.open(xfile)
colors = img.convert('RGB').getcolors() #this converts the mode to RGB

In [ ]:
print(colors)

In [ ]:
img

In [ ]:
from PIL import Image
# https://www.hackerearth.com/pt-br/practice/notes/extracting-pixel-values-of-an-image-in-python/

# getdata() to extract the pixel values. this scans the image horizontally from left to right starting at
# the top-left corner. The values got from each pixel is then added into a list.
# Finally what we get is a list with each pixel value as a set of 4 values(R,G,B.A).

pix_val = list(im.getdata())

In [ ]:
pix_val

In [ ]:
type(pix_val)

In [ ]:
# https://www.hackerearth.com/pt-br/practice/notes/extracting-pixel-values-of-an-image-in-python/
    
# pix_val is the list that contains all the pixel values which can be printed to see those values 
# But the list got is a list of sets and some times its needed to flatten the list for example if the 
# list is like: [(123,124,145,120), (345,453,234,124),……] 
# and the list that is needed is [123, 124, 145, 120, 345, 453, 234, 124….] then the command to flatten the list is:

pix_val_flat = [x for sets in pix_val for x in sets]

In [ ]:
pix_val_flat

In [ ]:
# Sum the list
xx = sum(pix_val_flat)

# https://convertingcolors.com/decimal-color-139.5525.html
# medium color
yy = xx / len(pix_val_flat)

In [ ]:
yy

In [ ]:
# Name colors
# http://www.discoveryplayground.com/computer-programming-for-kids/rgb-colors/
# https://matplotlib.org/examples/color/named_colors.html

In [ ]:
# https://matplotlib.org/gallery/color/named_colors.html
# Python from color name to RGB
from matplotlib import colors

print(colors.to_rgba('blue'))

In [ ]:
# https://matplotlib.org/2.0.0/examples/color/named_colors.html
from matplotlib import colors as mcolors


colors = dict(mcolors.BASE_COLORS, **mcolors.CSS4_COLORS)

# Sort colors by hue, saturation, value and name.
by_hsv = sorted((tuple(mcolors.rgb_to_hsv(mcolors.to_rgba(color)[:3])), name)
                for name, color in colors.items())
sorted_names = [name for hsv, name in by_hsv]


In [ ]:
colors